In [52]:
import numpy as np
import graphviz
from pprint import pprint

# Set the seed for random generations
np.random.seed(12345)

In [53]:
!dot -Tpng -O diagraph.txt

![diagraph.png](./diagraph.txt.png)

As a convention, J_ij is the link that goes from node j to node i

In [54]:
# I try to create a class for the NN
class Network:
    """
    Class for a generic neural network, defined by 
    - 2 input neurons, 1 output neuron
    - S: # of intra neurons.
    - Theta: threshold value.
    - J: connectivity matrix.
    """
    
    def __init__(self):
        self.S = 1      # Initialize with one intra neuron
        self.Theta = np.zeros(2+self.S+1)   # I instantiate a threshold value for each neuron, even the input, just for consistency of the indexes
        # Define the connectivity matrix J0_ij
        J0 = np.ones((2+self.S+1,2+self.S+1)) # Initialized all to one, so all connnected
        for j in range(len(J0)):            # First, impose on J0 that the elements on the diagonal must be zero
            J0[j,j] = 0.
        J0[0,1] = 0.            # Impose on the connectiviy matrix the fact that nuerons on the same layer cannot be connected 
        J0[1,0] = 0.
        for i in range(2, 2 + self.S):
            for j in range(2, 2 + self.S):
                if i != j:
                    J0[i, j] = 0.       
        self.J = J0
        self.activation = lambda x, theta: 0 if x < theta else 1    # activation function (if x = theta -> returns 1)
        self.neurons = np.zeros(2+self.S+1) # container for storing the values of the neurons
        self.fitness = 0.
        
    def initJ(self):            # QUESTO VA POI FATTO MEGLIO
        # Define the connectivity matrix J0_ij
        J0 = np.ones((2+self.S+1,2+self.S+1)) # Initialized all to one, so all connnected
        for j in range(len(J0)):            # First, impose on J0 that the elements on the diagonal must be zero
            J0[j,j] = 0.
        J0[0,1] = 0.            # Impose on the connectiviy matrix the fact that nuerons on the same layer cannot be connected 
        J0[1,0] = 0.
        for i in range(2, 2 + self.S):
            for j in range(2, 2 + self.S):
                if i != j:
                    J0[i, j] = 0. 
        return J0
    
    def generate(self,par:dict):
        """Generate a random network.

        Args:
            par (dict, optional): parameters of the generation.
        """
        self.S = par['Sk_range'][np.random.randint(len(par['Sk_range']))]   # Generate S
        self.Theta = np.random.uniform(par['Theta_range'][0],par['Theta_range'][1],2+self.S+1) # Generate the set of thresholds, Theta
        self.J = self.initJ() * np.random.uniform(par['J_range'][0],par['J_range'][1],(2+self.S+1)**2).reshape((2+self.S+1,2+self.S+1))   # Generate the connectivity matrix, J
        self.neurons = np.zeros(2+self.S+1)
        
    def ff(self,input:list,verb:int=0): # feed-forward
        """Compute the output of the network by feed-forward, given an input.

        Args:
            input (list): input to the network. Supported inputs are [0,0];[0,1];[1,0];[1,1].
            verb (int,optional): verbosity. If > 0, returns the value of each neuron. Default to 0.

        Returns:
            output (int): output of the network.
        """
        self.neurons[0:2] = input   # set the value of the first two neurons to the input value
        
        for neuron in range(1,2+self.S):    # compute the value of the intra neurons by feed forward
            self.neurons[neuron] = self.J[neuron,0] * self.neurons[0] + self.J[neuron,1] * self.neurons[1]   # I refer to the lower triangle of the matrix J
            self.neurons[neuron] = self.activation(self.neurons[neuron],self.Theta[neuron])     # Activation
        
        for neuron in range(0,2+self.S):  # compute the value of the ouput
            self.neurons[-1] = self.J[-1,neuron] * self.neurons[neuron]     # Add each neuron's weighted contribution to the output
        self.neurons[-1] = self.activation(self.neurons[-1],self.Theta[-1])     # activation
        output = self.neurons[-1]
        if verb > 0:
            return self.neurons
        return output
        
    def compute_fitness(self,par:dict):
        """Compute the output of the network and from that the fitness value 
            and with that updates the network's fitness value.

        Args:
            par (dict): parameters of the generation.
            
        """
        for i,input in enumerate(par['input_set']):
            output = self.ff(input)
            squared_dist = (par['target_set'][i] - output)**2     # square distance between network output and target (theoretical) output
            squared_cost = (np.sum([0 if np.isclose(j, 0., rtol=1e-3) else 1 for j in self.J.flatten()])/2)**2  # check if the tolerance is good
            self.fitness += squared_dist * squared_cost # add to the fitness value of the network
        self.fitness / len(par['input_set'])    # normalize over the inputs
    

For this model I don't need to define the space grid, I just need to generate the solutions and then execute the algorithm. I just need to make sure that they stay in the range.

I try first one loop:

In [55]:
# Define the set of parameters
par = {'Sk_range': [1,2,3,4,5],  # define the possible value for Sk
       'Theta_range': [-1,+1],     # define the possible value for Theta
       'J_range': [-1,+1],  # define the possible value for J
       'N_sol': 20,         # Set the number of solutions
       'input_set':[[0,0],[0,1],[1,0],[1,1]],    # Set of inputs
       'target_set': [0,1,1,0],    # set of outputs for the XOR gate 
       'mutation_ratio': 0.7,             # set the ratio of mutated solutions over the whole number of individuals 
                                          # that have to be generate after a selection. Who is not mutated is random 
       'S_mutation_radius': 1,
       'Theta_mutations_radius': 1/20,
       'J_mutations_radius': 1/20}                               

In [56]:
a = [0,0]
a = par['input_set'][3]
n = Network()
n.generate(par)
pprint(vars(n))


{'J': array([[-0.        ,  0.        , -0.25215765, -0.69005329,  0.78468746,
        -0.94642055],
       [-0.        , -0.        ,  0.61457742,  0.25418865,  0.81584979,
         0.11279461],
       [ 0.6798385 , -0.89902409,  0.        ,  0.        , -0.        ,
         0.38189585],
       [-0.74137086,  0.6653728 , -0.        ,  0.        ,  0.        ,
        -0.27258632],
       [ 0.58994114,  0.3969623 ,  0.        ,  0.        ,  0.        ,
         0.81991544],
       [ 0.94982718,  0.31223918,  0.62397913, -0.79451202, -0.52494309,
        -0.        ]]),
 'S': 3,
 'Theta': array([ 0.78030943, -0.73858541, -0.92048101,  0.65287226,  0.06415584,
        0.91261996]),
 'activation': <function Network.__init__.<locals>.<lambda> at 0x7f3127b77f60>,
 'fitness': 0.0,
 'neurons': array([0., 0., 0., 0., 0., 0.])}


In [49]:
a = np.array([1,2,4,5])
a = np.insert(a,obj=-1,values=3)
print(a)
np.delete(a,[1,2])

[1 2 4 3 5]


array([1, 3, 5])

In [51]:
np.linspace(1,5,5,dtype=int)

array([1, 2, 3, 4, 5])

In [ ]:
# Generate the solutions
solutions = []
for _ in range(par['N_sol']):
    # Generate a solution
    n = Network()
    n = n.generate(par)
    solutions.append(n)

# Compute the fitness value for each solution
fitness_values = []
for solution in range(par['N_sol']):
    solutions[solution].compute_fitness(par)
    fitness_values.append(solutions[solution].fitness)
    
# Compute the mean fitness
mean_fit = np.sum(fitness_values) / par['N_sol']    # Here I don't have to divide also by 4, 
                                                    # because I've already done it in the compute_fitness method

# discard elements in sol whose fitness value is below average
solutions = [s.fitness >= mean_fit for s in solutions]

# Compute the number of discarded elements, m
m = par['N_sol'] - len(solutions)

# Extract the parents between the survivors (here we can either take them random or take the fittest survivors)
# randomly
n_parents = np.floor(par['mutation_ratio'] * m)
parents_idx = np.random.randint(0,len(solutions),n_parents)
parents = solutions[parents_idx]

# Firstly, instantiate the offsprings as copies of the parents
offspring = parents

# Now, mutate S first
for child in offspring:
    sign_idx = np.random.randint(2)     # extract uniformly 0 or 1 for choosing the sign of the mutation
    sign_array = [-1,+1]                # this is used to apply the sign
    child.S += sign_array[sign_idx] * par['S_mutation_radius']
    # Check the boundary conditions for S
    if child.S < min(par['Sk_range']):
        child.S = min(par['Sk_range'])
    if child.S > max(par['Sk_range']):
        child.S = max(par['Sk_range'])

# Now mutate the thresholds - if S decreased, you mutate only the remaining neurons, else, you randomly generate the new theta(s)
for i,child in enumerate(offspring):
    # compute the difference between the orignal S and the mutated one
    deltaS = child.S - parents[i].S
    if deltaS > 0:
        new_thetas = np.random.uniform(par['Theta_range'][0],par['Theta_range'][1],deltaS)
        child.Theta = np.insert(child.Theta,obj=-1,values=new_thetas)   # add the newly generated thetas before the output neuron
    else:
        np.delete(child.Theta,np.linspace(2+parents[i].S,2+child.S,deltaS)) # delete the theta values of the deleted neurons

    # mutate theta (I mutate also the newly generated thetas)
    mutationTheta = np.random.uniform(par['Theta_range'][0]*par['Theta_mutation_radius'], 
                                  par['Theta_range'][1]*par['Theta_mutation_radius'], len(child.Theta))   # generate the mutation radius 
    child.Theta += mutationTheta
    # Check the boundary conditions for Theta
    for theta in child.Theta:
        if theta < min(par['Theta_range']):
            theta = min(par['Theta_range'])
        if theta > max(par['Theta_range']):
            theta = max(par['Theta_range'])
            
# Mutate J
for i,child in enumerate(offspring):
    # compute the difference between the original S and the mutated one
    deltaS = child.S - parents[i].S
    J0 = child.initJ()     # I utilize the method initJ() to create a new J given the new S
    if deltaS > 0:
        for _ in range(deltaS):
            j = np.random.uniform(par['J_range'][0],par['J_range'][1])  # generate a random value to be given to the new links
            child.J = np.insert(child.J,obj=-1,values=j,axis=0)    # insert a row of 1s (axis 0)
            child.J = np.insert(child.J,obj=-1,values=j,axis=1)    # insert a column of 1s (axis 1)
    else:
        for _ in range(deltaS):
            np.delete(child.J,np.linspace(2+parents[i].S,2+child.S,deltaS,axis=0)) # delete the J values of the deleted neurons on axis 0
            np.delete(child.J,np.linspace(2+parents[i].S,2+child.S,deltaS,axis=1)) # delete the J values of the deleted neurons on axis 1        
    # multiplicate with J0 to set to 0 where necessary by the conditions
    child.J *= J0
    # mutate (I mutate also the newly generated thetas)
    mutationJ = np.random.uniform(par['J_range'][0]*par['J_mutation_radius'], 
                                  par['J_range'][1]*par['J_mutation_radius'], len(child.J)**2)   # generate the mutation radius 
    # Check the boundary conditions for J
    for j in child.J:
        if j < min(par['J_range']):
            j = min(par['J_range'])
        if j > max(par['J_range']):
            j = max(par['J_range'])
            
# Fix also the other attributes
for i,child in enumerate(offspring):
    # compute the difference between the orignal S and the mutated one
    deltaS = child.S - parents[i].S
    if deltaS > 0:
        child.neurons = np.insert(child.neurons,obj=-1,values=0.5)      # i set the new neurons to 0.5
    else:
        np.delete(child.neurons,2+child.S,deltaS)   # delete the eliminated neurons

SyntaxError: invalid syntax (4096696115.py, line 13)